In [ ]:
pip install --upgrade pip

In [ ]:
#for saving the videos and animations used for visualisation
!pip install "imageio[ffmpeg]"


In [ ]:
#Allows interactive plotting 
!pip install ipympl


# libraries and dependencies

In [ ]:
import numpy as np                  #for numerical calculation
import matplotlib.pyplot as plt     #for plotting
from mpl_toolkits.mplot3d import Axes3D           #for 3D plotting
from matplotlib.animation import FuncAnimation    #for animation
from IPython.display import HTML                  #to display animation inline in the notebook

In [ ]:
#activates the interactive window in python Jupyter
%matplotlib widget


# Simulation Specifications

In [ ]:
L=10.0           #size of the cubic simulation box(L*L*L)
rc=2.5           #cutoff radius for LJ potential beyond which the potential is 0
V=1000.0         #volume of the simulation box
rho=0.1          #number density- no of particles per unit volume
N=int(rho*V)     #total number of particles
T=0.4            #Temperature 
beta=1.0/T       #inverse temp (for metropolis)
n_steps=20000    #no of monte carlo steps
max_disp=0.4     #max displacement for trial move
save_every=10    #save config everyt 10 steps for animation

np.random.seed(42)  #same randpmness everytime code is run

T = 0.3 means:

Temperature is low, relative to the depth of the potential well (ε). particles move slowly and perfer low potential

The system has less thermal energy compared to the potential energy barrier.

# Generates N random 3d postitons inside the box

In [ ]:
def initialise_positions(N,L):
    return np.random.rand(N,3)*L

# Ensures no overlap 

In [ ]:
def initialize_positions_no_overlap(N, L, min_dist=0.8):
    #blank list to later add to the code
    positions = []
    while len(positions) < N:
        
        #randomly generates trial position
        trial = np.random.rand(3) * L
        
        #checks if its min dist from all positions
        if all(np.linalg.norm(NearestImg(trial - np.array(pos), L)) > min_dist for pos in positions):
            positions.append(trial)
    return np.array(positions)


# Applies periodic boundary condition 

In [ ]:
def apply_periodic(pos,L):
    return pos%L

In [ ]:
#finds the shortest distance between two particles, accounting for periodic boundaries.
def NearestImg(rij,L):    
    return rij-L*np.round(rij/L)

# Energy and potential calculation

In [ ]:
#lenard jones potential
def lj_potential(r2):
    if r2 < rc**2:
        inv_r6 = (1.0 / r2)**3
        inv_r12 = inv_r6**2
        return 4 * (inv_r12 - inv_r6)
    return 0.0

In [ ]:
#calculate total energy using LJ potential
def total_energy(positions):
    energy = 0.0
    for i in range(N):
        for j in range(i + 1, N):
            rij = NearestImg(positions[i] - positions[j], L)
            r2 = np.dot(rij, rij)
            energy += lj_potential(r2)
    return energy

# Part where the the random displacement and energy diff calculatiosna re performed


# Monte Carlo Step

In [ ]:
def monte_carlo_step(positions, energy):
    #random particle selection of particle
    i = np.random.randint(N)
    old_pos = positions[i].copy()
    new_pos = old_pos + (np.random.rand(3) - 0.5) * max_disp
    new_pos = apply_periodic(new_pos, L)

    #energy diff calculation
    dE = 0.0
    for j in range(N):
        if j != i:
            rij_old = NearestImg(old_pos - positions[j], L)
            rij_new = NearestImg(new_pos - positions[j], L)
            r2_old = np.dot(rij_old, rij_old)
            r2_new = np.dot(rij_new, rij_new)
            dE += lj_potential(r2_new) - lj_potential(r2_old)

    #acceptance critera        
    if dE < 0.0 or np.random.rand() < np.exp(-beta * dE):
        positions[i] = new_pos
        energy += dE
    return positions, energy

# Here the positions are updated based on Energy diff calculation

# Main Simulation Loop

In [ ]:
positions = initialize_positions_no_overlap(N, L)

energy = total_energy(positions)
trajectory = []
energies = []

#main simulation loop
for step in range(n_steps):
    
    #monte carlo step
    positions, energy = monte_carlo_step(positions, energy)
    if step % save_every == 0:
        trajectory.append(positions.copy())
        
        #saves the current total energy
        energies.append(energy)


In [ ]:
accepted_moves = 0
total_moves = 0

trajectory = []   # List to store configurations
energies = []     # List to store energy values


def monte_carlo_step(positions, energy):
    i = np.random.randint(N)
    old_pos = positions[i].copy()
    new_pos = old_pos + (np.random.rand(3) - 0.5) * max_disp
    new_pos = apply_periodic(new_pos, L)

    dE = 0.0
    for j in range(N):
        if j != i:
            rij_old = NearestImg(old_pos - positions[j], L)
            rij_new = NearestImg(new_pos - positions[j], L)
            r2_old = np.dot(rij_old, rij_old)
            r2_new = np.dot(rij_new, rij_new)
            dE += lj_potential(r2_new) - lj_potential(r2_old)

    accept = False
    if dE < 0.0 or np.random.rand() < np.exp(-beta * dE):
        positions[i] = new_pos
        energy += dE
        accept = True

    return positions, energy, accept

for step in range(n_steps):
    positions, energy, accepted = monte_carlo_step(positions, energy)
    total_moves += 1
    if accepted:
        accepted_moves += 1

    if step % save_every == 0:
        trajectory.append(positions.copy())
        energies.append(energy)
        
acceptance_ratio = accepted_moves / total_moves
print(f"Acceptance Ratio: {acceptance_ratio:.3f}")

# Visualisations

# Energy Plot

In [ ]:
plt.figure(figsize=(6,4))
plt.plot(range(0, n_steps, save_every), energies)
plt.xlabel('Steps')
plt.ylabel('Total Energy')
plt.title('Energy over Time')
plt.grid(True)
plt.tight_layout()
plt.show()


Initial High Energy: At the beginning of the simulation, the energy will likely be high due to random particle placements.

Decrease Over Time: After a number of steps, the energy should decrease as particles move toward more favorable positions with minimal interaction energy. This trend could stabilize over time.

Stable Energy: After some Monte Carlo steps (likely a few thousand), the energy might reach a relatively stable value, signifying that the system has equilibrated.

# Energy Histogram- assuming equilibrium

In [ ]:
trajectory = np.array(trajectory)  # Convert list to ndarray
n_frames, n_particles, _ = trajectory.shape


In [ ]:
import matplotlib as mpl
mpl.rcParams['animation.embed_limit'] = 100  # allows ~100MB instead of 20MB


In [ ]:
from IPython.display import display
import ipywidgets as widgets

trajectory = np.array(trajectory)  # Ensure it's a NumPy array
n_frames, n_particles, _ = trajectory.shape

initial_positions = trajectory[0]

# --- 2 Subplots ---
fig = plt.figure(figsize=(24, 10))

# 3D particle movement
ax1 = fig.add_subplot(121, projection='3d')
scat = ax1.scatter([], [], [], s=10, c='red')
trail_lines = [ax1.plot([], [], [], color='lightcoral', linewidth=0.5)[0] for _ in range(n_particles)]

ax1.set_xlim(0, L)
ax1.set_ylim(0, L)
ax1.set_zlim(0, L)
ax1.set_title('3D Particle Movement')

# 2D displacement graph
ax2 = fig.add_subplot(122)
line2d, = ax2.plot([], [], lw=2)
ax2.set_xlim(0, n_frames)
ax2.set_ylim(0, np.linalg.norm(trajectory - initial_positions, axis=2).max())
ax2.set_title('Total Displacement over Time')
ax2.set_xlabel('Frame')
ax2.set_ylabel('Displacement')

# Store displacement over time
displacements = np.linalg.norm(trajectory - initial_positions, axis=2).mean(axis=1)  # (n_frames,)

# --- Animation Function ---
def update(frame):
    pos = trajectory[frame]
    
    # Update 3D positions
    scat._offsets3d = (pos[:, 0], pos[:, 1], pos[:, 2])
    
    for i, trail in enumerate(trail_lines):
        trail.set_data(trajectory[:frame+1, i, 0], trajectory[:frame+1, i, 1])
        trail.set_3d_properties(trajectory[:frame+1, i, 2])
    
    # Update 2D plot
    line2d.set_data(np.arange(frame+1), displacements[:frame+1])
    
    return scat, *trail_lines, line2d

# Sliders
frame_slider = widgets.IntSlider(min=0, max=n_frames-1, step=1, description='Frame')
speed_slider = widgets.IntSlider(min=10, max=200, step=10, value=50, description='Speed')

# Animation control
ani = FuncAnimation(fig, update, frames=n_frames, interval=speed_slider.value, blit=False)
display(widgets.VBox([frame_slider, speed_slider]))

from IPython.display import HTML
HTML(ani.to_jshtml())


In [ ]:
# Convert energies list to numpy array (in case it's a list)
energy_array = np.array(energies)
n_frames = len(energy_array)

# Set up the plot
fig, ax = plt.subplots(figsize=(8, 5))
ax.set_xlim(0, n_frames)
ax.set_ylim(energy_array.min() - 5, energy_array.max() + 5)
ax.set_title('Energy over Time')
ax.set_xlabel('Steps (x{})'.format(save_every))  # Show actual steps
ax.set_ylabel('Total Energy')

# Line and moving dot
line, = ax.plot([], [], color='red', lw=2)
dot, = ax.plot([], [], 'ro')

# Update function for animation
def update(frame):
    x = np.arange(frame + 1)
    y = energy_array[:frame + 1]
    line.set_data(x, y)
    dot.set_data(frame, energy_array[frame])
    return line, dot

# Create animation
ani = FuncAnimation(fig, update, frames=n_frames, interval=50, blit=True)
HTML(ani.to_jshtml())


In [ ]:

for idx, e in enumerate(energies):
    print(f"Step {idx * save_every}: Energy = {e:.3f}")
